In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [93]:
from structs import Results

TEST_TIMES = 100

model_results = {}
target_col = ["Attrition"]


In [94]:
df = pd.read_csv("rs6-attrition-predict/train.csv")

pd.options.display.max_columns = None

display(df)

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,1938,4,Female,72,3,4,Manager,4,Married,17875,11761,4,Y,Yes,13,3,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,1546,4,Male,97,3,1,Research Scientist,4,Married,2132,4585,4,Y,No,20,4,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,1066,3,Male,74,3,2,Sales Executive,1,Married,8396,22217,1,Y,No,14,3,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,789,1,Male,76,2,3,Sales Executive,1,Single,7587,14229,1,Y,No,15,3,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,1273,3,Male,73,1,1,Sales Representative,4,Single,1118,8040,1,Y,Yes,14,3,4,80,0,1,4,3,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,763,34,No,Travel_Rarely,1333,Sales,10,4,Life Sciences,1,1055,3,Female,87,3,1,Sales Representative,3,Married,2220,18410,1,Y,Yes,19,3,4,80,1,1,2,3,1,1,0,0
1172,835,35,No,Travel_Rarely,528,Human Resources,8,4,Technical Degree,1,1164,3,Male,100,3,1,Human Resources,3,Single,4323,7108,1,Y,No,17,3,2,80,0,6,2,1,5,4,1,4
1173,1216,43,No,Travel_Rarely,1179,Sales,2,3,Medical,1,1706,4,Male,73,3,2,Sales Executive,4,Married,7847,6069,1,Y,Yes,17,3,1,80,1,10,3,3,10,9,8,8
1174,559,38,No,Travel_Rarely,268,Research & Development,2,5,Medical,1,773,4,Male,92,3,1,Research Scientist,3,Married,3057,20471,6,Y,Yes,13,3,2,80,1,6,0,1,1,0,0,1


In [95]:
for col in df.columns:
	if df[col].isna().sum() > 0:
		print(col, df[col].isna().sum())

In [96]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

hot_enc = OneHotEncoder(sparse_output = False, handle_unknown = "ignore")
label_enc = LabelEncoder()

drop_columns = ["user_id", "EmployeeCount", "EmployeeNumber", "Over18", "StandardHours", "Gender", "RelationshipSatisfaction", "EducationField",  "PerformanceRating", "JobRole", "Department"]
hot_columns = ["MaritalStatus", "BusinessTravel", "WorkLifeBalance"]
label_columns = ["Attrition", "OverTime"]

try:
	df = df.drop(columns = drop_columns)
	hot_array = hot_enc.fit_transform(df[hot_columns])
	df = df.drop(columns = hot_columns)

	df = pd.concat([df, pd.DataFrame(hot_array, columns = hot_enc.get_feature_names_out(hot_columns))], axis = 1)
 
	for col in label_columns:
		df[col] = label_enc.fit_transform(df[col])
except KeyError:
    pass

df.head(5)

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,WorkLifeBalance_1,WorkLifeBalance_2,WorkLifeBalance_3,WorkLifeBalance_4
0,58,0,605,21,3,4,72,3,4,4,17875,11761,4,1,13,1,29,2,1,0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,45,0,950,28,3,4,97,3,1,4,2132,4585,4,0,20,1,8,3,5,4,0,3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,40,0,300,26,3,3,74,3,2,1,8396,22217,1,0,14,1,8,3,7,7,7,5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,36,0,1434,8,4,1,76,2,3,1,7587,14229,1,0,15,0,10,1,10,7,0,9,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,25,1,599,24,1,3,73,1,1,4,1118,8040,1,1,14,0,1,4,1,0,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [97]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

X = df.drop(columns = target_col)
y = df["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [98]:
from sklearn.svm import SVC

def get_best_svc(X_train, y_train):

	svc = SVC(class_weight="balanced")

	grid = {
		"kernel": ["linear", "rbf"],
		"C": [0.1, 1, 10, 50, 100],
		"gamma": ["scale", "auto"],
	}

	svc_grid = GridSearchCV(estimator = svc, param_grid = grid, cv = 8, verbose = 2, n_jobs = -1)

	X_train_scaled = scaler.transform(X_train)
	svc_grid.fit(X_train_scaled, y_train)

	print(f"SVC Best Params: {svc_grid.best_params_}\n")

	return svc_grid.best_estimator_

In [99]:
from sklearn.ensemble import RandomForestClassifier

def get_best_rf(X_train, y_train):

	rf = RandomForestClassifier(class_weight="balanced")

	grid = {
		"n_estimators": [60, 70, 80, 90, 100],
		"max_features": [2, 3, "sqrt", "log2"],
		"max_depth": [100, 200, 250, 300, 350],
		"min_samples_split": [9, 10, 11, 12],
		"min_samples_leaf": [1, 2, 3],
		'bootstrap': [True],
	}

	rf_grid = GridSearchCV(estimator = rf, param_grid = grid, cv = 8, verbose = 2, n_jobs = -1)
 
	X_train_scaled = scaler.transform(X_train)
	rf_grid.fit(X_train_scaled, y_train)

	print(f"Random Forest Best Params: {rf_grid.best_params_}\n")

	return rf_grid.best_estimator_

In [100]:
from sklearn.neighbors import KNeighborsClassifier

def get_best_knn(X_train, y_train):
    
    knn = KNeighborsClassifier()
    
    grid = {
		"n_neighbors": list(range(8, 15))
	}
    
    knn_grid = GridSearchCV(estimator = knn, param_grid = grid, cv = 8, verbose = 2, n_jobs = -1)
    
    X_train_scaled = scaler.transform(X_train)
    knn_grid.fit(X_train_scaled, y_train)
    
    print(f"K Nearest Neighbors Best Params: {knn_grid.best_params_}\n")
    
    return knn_grid.best_estimator_

In [101]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score, accuracy_score, precision_score

def generate_results(results: Results, model: object, X, y, test_times = 10, set_random = 42):
    for _ in range(test_times):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = set_random)
        X_test_scaled = scaler.transform(X_test)
        X_train_scaled = scaler.transform(X_train)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        try:
            results.importances.append(model.feature_importances_)
        except AttributeError:
            results.importances.append(0)
    
        results.accuracies.append(accuracy_score(y_test, y_pred))
        results.precisions.append(precision_score(y_test, y_pred))
        results.recalls.append(recall_score(y_test, y_pred))
        results.conf_matrices.append(confusion_matrix(y_test, y_pred))
        results.f1_scores.append(f1_score(y_test, y_pred))

In [102]:
def print_results(results_dictionary: dict, prepend = False):
    if prepend:
        print(prepend)
    for key, value in results_dictionary.items():
        if type(value) == np.float64:
            print(f"{key}: {(value * 100).round(2)}%")
        else:
            print(f"{key}: {value} Type: {type(value)}")
    print()

In [103]:
RF_Results = Results("RandomForest")
SVC_Results = Results("SVC")
KNN_Results = Results("KNN")

generate_results(RF_Results, get_best_rf(X_train, y_train), X, y, TEST_TIMES)
generate_results(SVC_Results, get_best_svc(X_train, y_train), X, y, TEST_TIMES)
generate_results(KNN_Results, get_best_knn(X_train, y_train), X, y, TEST_TIMES)

rf_averages: dict = RF_Results.calculate_dictionary("mean")
rf_medians: dict = RF_Results.calculate_dictionary("median")

svc_averages: dict = SVC_Results.calculate_dictionary("mean")
svc_medians: dict = SVC_Results.calculate_dictionary("median")

knn_averages: dict = KNN_Results.calculate_dictionary("mean")
knn_medians: dict = KNN_Results.calculate_dictionary("median")

print_results(rf_averages, "Random Forest Averages: ")
print_results(svc_averages, "SVC Averages: ")
print_results(knn_averages, "KNN Averages: ")

Fitting 8 folds for each of 1200 candidates, totalling 9600 fits
Random Forest Best Params: {'bootstrap': True, 'max_depth': 350, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 60}

Fitting 8 folds for each of 20 candidates, totalling 160 fits
SVC Best Params: {'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}

Fitting 8 folds for each of 30 candidates, totalling 240 fits
K Nearest Neighbors Best Params: {'n_neighbors': 11}

Random Forest Averages: 
importances: 3.23%
accuracies: 83.3%
precisions: 72.24%
recalls: 31.98%
f1_scores: 44.2%
conf_matrices: 5900.0%

SVC Averages: 
importances: 0.0%
accuracies: 81.36%
precisions: 58.06%
recalls: 36.73%
f1_scores: 45.0%
conf_matrices: 5900.0%

KNN Averages: 
importances: 0.0%
accuracies: 79.66%
precisions: 66.67%
recalls: 4.08%
f1_scores: 7.69%
conf_matrices: 5900.0%

